In [6]:
from azureml.core import Workspace,Dataset,Experiment
from azureml.train.automl import AutoMLConfig
from azureml.widgets import RunDetails 
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,confusion_matrix

In [7]:
ws=Workspace.from_config()

In [8]:
ws

Workspace.create(name='windia', subscription_id='cfa28f36-8345-475f-aa77-126e68d22b1e', resource_group='mam')

In [9]:
dataset=Dataset.get_by_name(workspace=ws,name='mydataz')

In [10]:
dataset

{
  "source": [
    "('workspaceblobstore', 'UI/05-22-2021_101317_UTC/diabetes.csv')"
  ],
  "definition": [
    "GetDatastoreFiles",
    "ParseDelimited",
    "DropColumns",
    "SetColumnTypes"
  ],
  "registration": {
    "id": "3a1aa232-c507-450b-a6c8-07df8032e409",
    "name": "mydataz",
    "version": 1,
    "workspace": "Workspace.create(name='windia', subscription_id='cfa28f36-8345-475f-aa77-126e68d22b1e', resource_group='mam')"
  }
}

In [11]:
diabetes_df=dataset.to_pandas_dataframe()

In [12]:
diabetes_df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [13]:
x_train,x_test=train_test_split(diabetes_df,test_size=0.1)

In [23]:
automl_settings={
    "iteration_timeout_minutes":2,
    "experiment_timeout_minutes":15,
    "enable_early_stopping":True,
    "primary_metric":"accuracy",
    "featurization":"auto",
    "n_cross_validation":5
}

In [24]:
automl_config=AutoMLConfig(task="classification",debug_log="automl_errors.log",training_data=x_train,label_column_name="Outcome",**automl_settings)

In [25]:
experiment=Experiment(ws,"diabetec_experiment")

In [26]:
run=experiment.submit(automl_config,show_output=True)

No run_configuration provided, running on local with default configuration
Running in the active local environment.


Experiment,Id,Type,Status,Details Page,Docs Page
diabetec_experiment,AutoML_956c9abb-d368-4842-823b-1b9394d7303c,automl,Preparing,Link to Azure Machine Learning studio,Link to Documentation


Current status: DatasetEvaluation. Gathering dataset statistics.
Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetFeaturizationCompleted. Completed fit featurizers and featurizing the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values

****************************************************************************************************
Current status: BestRunExplainModel. Best run model explanations started
Current status: ModelExplanationDataSetSetup. Model explanations data setup completed
Current status: PickSurrogateModel. Choosing LightGBM as the surrogate model for explanations
Current status: EngineeredFeatureExplanations. Computation of engineered features started
Current status: EngineeredFeatureExplanations. Computation of engineered features completed
Current status: RawFeaturesExplanations. Computation of raw features started
Current status: RawFeaturesExplanations. Computation of raw features completed
Current status: BestRunExplainModel. Best run model explanations completed
****************************************************************************************************


In [27]:
RunDetailstails(run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [28]:
best_run,fitted_model=run.get_output()

In [29]:
print(best_run)

Run(Experiment: diabetec_experiment,
Id: AutoML_956c9abb-d368-4842-823b-1b9394d7303c_36,
Type: None,
Status: Completed)


In [30]:
print(fitted_model)

Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('prefittedsoftvotingclassifier',...
                                                                                                reg_lambda=0.42105263157894735,
                                                                                                silent=True,
                                                                                                subsample=0.8910526315789474,
                             

In [35]:
x_test=x_test.drop("Outcome")

KeyError: "['Outcome'] not found in axis"

In [36]:
y_predict=fitted_model.predict(x_test)

In [39]:
y_predict

array([0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1,
       1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0,
       1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0])